In [1]:
import plotly.express as px
from scipy.stats import ttest_ind

In [2]:
from cpa_utils import *

In [3]:
traces_path = "..\\acquisition\\carto_eB4-Rnd-3-WhiteningAndFullFilter-125000_samples_alternating_same_and_varying_fixed_second_round\\carto_eB4-Rnd-3-WhiteningAndFullFilter-Alternating-Same-And-Varying-Seeds.mat"
key_path = "..\\acquisition\\carto_eB4-Rnd-3-WhiteningAndFullFilter-125000_samples_alternating_same_and_varying_fixed_second_round\\carto_eB4-Rnd-3-WhiteningAndFullFilter-Alternating-Same-And-Varying-Seeds.log"

In [4]:
seeds_same, traces_same, seeds_varying, traces_varying, real_keys = load_data_alternating_same_varying(traces_path, key_path)

In [5]:
t_test = ttest_ind(traces_same[0], traces_varying[0], axis=0, equal_var=False)

In [6]:
np.where(np.abs(t_test.statistic) > 4.5)[0].shape[0]

13963

In [7]:
t_test.pvalue.shape[0]

50002

In [ ]:
def hypothesis_b4_rws_x_i(iv: str, key: List[int], round_idx: int, block_idx: int) -> int:
    indices, whitening = chacha_random_b4(iv)

    block = [(key[indices[i]] + whitening[i]) % 16 for i in range(BLOCK_WIDTH_B4 * round_idx, BLOCK_WIDTH_B4 * (round_idx + 1))]
    
    return HW[block[block_idx]]

hw_x_i_locations = [[[0] * 10] * BLOCK_WIDTH_B4] * (KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4 - 2)
for round_idx in range(len(hw_x_i_locations), KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4):
    corr_round = []
    for block_idx in range(BLOCK_WIDTH_B4):
        hyps = np.array([hypothesis_b4_rws_x_i(iv, key, round_idx, block_idx) for i, key in enumerate(real_keys) for iv in seeds_varying[i]])
        corr = corr_coef(hyps, traces_varying.reshape((-1, traces_varying.shape[2])))
        loc = np.argmax(corr)
        corr_round.append(list(range(loc - 5, loc + 5)))
        plt.plot(corr)
        plt.ylim([-0.5, 0.5])
        plt.title(f"Round {round_idx}, Block {block_idx}, Location: {loc}")
        plt.show()
    hw_x_i_locations.append(corr_round)
hw_x_i_locations = np.array(hw_x_i_locations)


In [ ]:
def hypothesis_b4_rws_x_i_diff(iv: str, key: List[int], round_idx: int, block_idx: int) -> int:
    indices, whitening = chacha_random_b4(iv)

    block = [(key[indices[i]] + whitening[i]) % 16 for i in range(BLOCK_WIDTH_B4 * round_idx, BLOCK_WIDTH_B4 * (round_idx + 1))]
    
    return HD[block[block_idx + 1]][block[block_idx]]

hw_x_i_diff_locations = [[[0] * 10] * (BLOCK_WIDTH_B4 - 1)] * (KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4 - 2)
for round_idx in range(len(hw_x_i_diff_locations), KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4):
    corr_round = []
    for block_idx in range(BLOCK_WIDTH_B4 - 1):
        hyps = np.array([hypothesis_b4_rws_x_i_diff(iv, key, round_idx, block_idx) for i, key in enumerate(real_keys) for iv in seeds_varying[i]])
        corr = corr_coef(hyps, traces_varying.reshape((-1, traces_varying.shape[2])))
        loc = np.argmax(corr)
        corr_round.append(list(range(loc - 5, loc + 5)))
        plt.plot(corr)
        plt.ylim([-0.5, 0.5])
        plt.title(f"Round {round_idx}, Block {block_idx}, Location: {loc}")
        plt.show()
    hw_x_i_diff_locations.append(corr_round)
hw_x_i_diff_locations = np.array(hw_x_i_diff_locations)


In [ ]:
def hypothesis_b4_rws_preliminary_additions(iv: str, key: List[int], round_idx: int, block_indices: Tuple[int, int]) -> int:
    block_idx_1, block_idx_2 = block_indices
    if block_idx_1 != block_idx_2 - 1 or block_idx_2 % 2 != 1 or block_idx_2 >= BLOCK_WIDTH_B4:
        raise ValueError
    indices, whitening = chacha_random_b4(iv)

    block = [(key[indices[i]] + whitening[i]) % 16 for i in range(BLOCK_WIDTH_B4 * round_idx, BLOCK_WIDTH_B4 * (round_idx + 1))]

    return HD[block[block_idx_2] + block[block_idx_1]][block[block_idx_2]]

block_additions_locations = [[[0] * 10] * (BLOCK_WIDTH_B4 // 2)] * (KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4 - 2)
for round_idx in range(len(block_additions_locations), KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4):
    corr_round = []
    for block_idx_1, block_idx_2 in zip(range(0, BLOCK_WIDTH_B4, 2), range(1, BLOCK_WIDTH_B4, 2)):
        hyps = np.array([hypothesis_b4_rws_preliminary_additions(iv, key, round_idx, (block_idx_1, block_idx_2)) for i, key in enumerate(real_keys) for iv in seeds_varying[i]])
        corr = corr_coef(hyps, traces_varying.reshape((-1, traces_varying.shape[2])))
        if round_idx == 12:
            loc = 4000 + np.argmax(corr[4000:5300])
        elif round_idx == 13:
            loc = 25800 + np.argmax(corr[25800:27200])
        else:
            raise ValueError("Should not happen")
        corr_round.append(list(range(loc - 5, loc + 5)))
        plt.plot(corr)
        plt.ylim([-0.5, 0.5])
        plt.title(f"Round {round_idx}, Blocks {(block_idx_1, block_idx_2)}, Location: {loc}")
        plt.xlim([4000, 5300] if round_idx == 12 else [25800, 27200])
        plt.show()
    block_additions_locations.append(corr_round)
block_additions_locations = np.array(block_additions_locations)

In [ ]:
def hypothesis_b4_rws_x_i_during_first_sbox(iv: str, key: List[int], round_idx: int, block_idx: int) -> int:
    indices, whitening = chacha_random_b4(iv)

    block = [(key[indices[i]] + whitening[i]) % 16 for i in range(BLOCK_WIDTH_B4 * round_idx, BLOCK_WIDTH_B4 * (round_idx + 1))]
    
    return HW[block[block_idx]]

hw_x_i_during_sbox_locations = [[[0] * 10] * (BLOCK_WIDTH_B4 - 1)] * (KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4 - 2)
for round_idx in range(len(hw_x_i_during_sbox_locations), KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4):
    corr_round = []
    for block_idx in range(BLOCK_WIDTH_B4 - 1):
        hyps = np.array([hypothesis_b4_rws_x_i_during_first_sbox(iv, key, round_idx, block_idx) for i, key in enumerate(real_keys) for iv in seeds_varying[i]])
        corr = corr_coef(hyps, traces_varying.reshape((-1, traces_varying.shape[2])))
        if round_idx == 12:
            loc = 4500 + np.argmax(corr[4500:9500])
        elif round_idx == 13:
            loc = 26500 + np.argmax(corr[26500:31500])
        else:
            raise ValueError("Should not happen")
        corr_round.append(list(range(loc - 5, loc + 5)))
        plt.plot(corr)
        plt.ylim([-0.5, 0.5])
        plt.title(f"Round {round_idx}, Block {block_idx}, Location: {loc}")
        plt.xlim([4500, 9500] if round_idx == 12 else [26500, 31500])
        plt.show()
    hw_x_i_during_sbox_locations.append(corr_round)
hw_x_i_during_sbox_locations = np.array(hw_x_i_during_sbox_locations)


In [ ]:
def hypothesis_b4_rws_sboxes(iv: str, key: List[int], round_idx: int, block_idx: int) -> int:
    indices, whitening = chacha_random_b4(iv)

    block = [(key[indices[i]] + whitening[i]) % 16 for i in range(BLOCK_WIDTH_B4 * round_idx, BLOCK_WIDTH_B4 * (round_idx + 1))]

    if block_idx % 2 == 0:
        sbox_out = s_boxes_b4[block_idx][block[block_idx]]
    else:
        sbox_out = s_boxes_b4[block_idx][(block[block_idx] + block[block_idx - 1]) % 16]
    return HW[sbox_out]

sbox_output_locations = [[[0] * 10] * (BLOCK_WIDTH_B4 - 1)] * (KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4 - 2)
for round_idx in range(len(sbox_output_locations), KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4):
    corr_round = []
    for block_idx in range(BLOCK_WIDTH_B4 - 1):
        hyps = np.array([hypothesis_b4_rws_sboxes(iv, key, round_idx, block_idx) for i, key in enumerate(real_keys) for iv in seeds_varying[i]])
        corr = corr_coef(hyps, traces_varying.reshape((-1, traces_varying.shape[2])))
        loc = np.argmax(corr)
        corr_round.append(list(range(loc - 5, loc + 5)))
        plt.plot(corr)
        plt.ylim([-0.5, 0.5])
        plt.title(f"Round {round_idx}, Block {block_idx}, Location: {loc}")
        plt.show()
    sbox_output_locations.append(corr_round)
sbox_output_locations = np.array(sbox_output_locations)


In [ ]:
def hypothesis_b4_rws_z_2_i_2_i_1(iv: str, key: List[int], round_idx: int, block_idx: int) -> int:
    indices, whitening = chacha_random_b4(iv)

    block = [(key[indices[i]] + whitening[i]) % 16 for i in range(BLOCK_WIDTH_B4 * round_idx, BLOCK_WIDTH_B4 * (round_idx + 1))]
    for i in range(3):
        block[2*i + 1] = (block[2*i + 1] + block[2*i]) % 16
    y = [s_boxes_b4[i][block[i]] for i in range(BLOCK_WIDTH_B4 - 1)]

    if block_idx % 2 == 0 :
        return HW[(y[(block_idx - 1) % 6] + y[block_idx]) % 16]
    else:
        return HW[(y[(block_idx + 3) % 6] + y[block_idx]) % 16]

z_2_i_2_i_1_locations = [[[0] * 10] * (BLOCK_WIDTH_B4 - 1)] * (KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4 - 2)
for round_idx in range(len(z_2_i_2_i_1_locations), KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4):
    corr_round = []
    for block_idx in range(BLOCK_WIDTH_B4 - 1):
        hyps = np.array([hypothesis_b4_rws_z_2_i_2_i_1(iv, key, round_idx, block_idx) for i, key in enumerate(real_keys) for iv in seeds_varying[i]])
        corr = corr_coef(hyps, traces_varying.reshape((-1, traces_varying.shape[2])))
        loc = np.argmax(corr)
        corr_round.append(list(range(loc - 5, loc + 5)))
        plt.plot(corr)
        plt.ylim([-0.5, 0.5])
        plt.title(f"Round {round_idx}, Block {block_idx}, Location: {loc}")
        plt.show()

    z_2_i_2_i_1_locations.append(corr_round)
z_2_i_2_i_1_locations = np.array(z_2_i_2_i_1_locations)


In [ ]:
def hypothesis_b4_rws_add_pre_sboxes_second_round(iv: str, key: List[int], round_idx: int, block_idx: int) -> int:
    indices, whitening = chacha_random_b4(iv)

    block = [(key[indices[i]] + whitening[i]) % 16 for i in range(BLOCK_WIDTH_B4 * round_idx, BLOCK_WIDTH_B4 * (round_idx + 1))]
    for i in range(3):
        block[2*i + 1] = (block[2*i + 1] + block[2*i]) % 16
    y = [s_boxes_b4[i][block[i]] for i in range(BLOCK_WIDTH_B4 - 1)]
    z = [0] * 6
    for i in range(3):
        z[2*i] = (y[(2*i - 1) % 6] + y[2*i]) % 16
        z[2*i + 1] = (y[(2*i + 4) % 6] + y[2*i + 1]) % 16
    
    return HW[z[block_idx] + block[(block_idx + 2) % 6]]

add_pre_second_sbox_output_locations = [[[0] * 10] * (BLOCK_WIDTH_B4 - 1)] * (KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4 - 2)
for round_idx in range(len(add_pre_second_sbox_output_locations), KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4):
    corr_round = []
    for block_idx in range(BLOCK_WIDTH_B4 - 1):
        hyps = np.array([hypothesis_b4_rws_add_pre_sboxes_second_round(iv, key, round_idx, block_idx) for i, key in enumerate(real_keys) for iv in seeds_varying[i]])
        corr = corr_coef(hyps, traces_varying.reshape((-1, traces_varying.shape[2])))
        if round_idx == 12:
            loc = 11000 + np.argmax(corr[11000:16600])
        elif round_idx == 13:
            loc = 32900 + np.argmax(corr[32900:38500])
        else:
            raise ValueError("Should not happen")
        corr_round.append(list(range(loc - 5, loc + 5)))
        plt.plot(corr)
        plt.ylim([-0.5, 0.5])
        plt.title(f"Round {round_idx}, Block {block_idx}, Location: {loc}")
        plt.xlim([11000, 16600] if round_idx == 12 else [32900, 38500])
        plt.show()
    add_pre_second_sbox_output_locations.append(corr_round)
add_pre_second_sbox_output_locations = np.array(add_pre_second_sbox_output_locations)


In [ ]:
def hypothesis_b4_rws_sboxes_second_round(iv: str, key: List[int], round_idx: int, block_idx: int) -> int:
    indices, whitening = chacha_random_b4(iv)

    block = [(key[indices[i]] + whitening[i]) % 16 for i in range(BLOCK_WIDTH_B4 * round_idx, BLOCK_WIDTH_B4 * (round_idx + 1))]
    for i in range(3):
        block[2*i + 1] = (block[2*i + 1] + block[2*i]) % 16
    y = [s_boxes_b4[i][block[i]] for i in range(BLOCK_WIDTH_B4 - 1)]
    z = [0] * 6
    for i in range(3):
        z[2*i] = (y[(2*i - 1) % 6] + y[2*i]) % 16
        z[2*i + 1] = (y[(2*i + 4) % 6] + y[2*i + 1]) % 16
    
    for i in range(6):
        z[i] = (z[i] + block[(i + 2) % 6]) % 16
        z[i] = s_boxes_b4[i + 6][z[i]]

    return HW[z[block_idx]]

second_sbox_output_locations = [[[0] * 10] * (BLOCK_WIDTH_B4 - 1)] * (KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4 - 2)
for round_idx in range(len(second_sbox_output_locations), KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4):
    corr_round = []
    for block_idx in range(BLOCK_WIDTH_B4 - 1):
        hyps = np.array([hypothesis_b4_rws_sboxes_second_round(iv, key, round_idx, block_idx) for i, key in enumerate(real_keys) for iv in seeds_varying[i]])
        corr = corr_coef(hyps, traces_varying.reshape((-1, traces_varying.shape[2])))
        loc = np.argmax(corr)
        corr_round.append(list(range(loc - 5, loc + 5)))
        plt.plot(corr)
        plt.ylim([-0.5, 0.5])
        plt.title(f"Round {round_idx}, Block {block_idx}, Location: {loc}")
        plt.show()
    second_sbox_output_locations.append(corr_round)
second_sbox_output_locations = np.array(second_sbox_output_locations)


In [ ]:
def hypothesis_b4_rws_t_i_store(iv: str, key: List[int], round_idx: int, block_idx: int) -> int:
    indices, whitening = chacha_random_b4(iv)

    block = [(key[indices[i]] + whitening[i]) % 16 for i in range(BLOCK_WIDTH_B4 * round_idx, BLOCK_WIDTH_B4 * (round_idx + 1))]
    for i in range(3):
        block[2*i + 1] = (block[2*i + 1] + block[2*i]) % 16
    y = [s_boxes_b4[i][block[i]] for i in range(BLOCK_WIDTH_B4 - 1)]
    z = [0] * 6
    for i in range(3):
        z[2*i] = (y[(2*i - 1) % 6] + y[2*i]) % 16
        z[2*i + 1] = (y[(2*i + 4) % 6] + y[2*i + 1]) % 16
    
    for i in range(6):
        z[i] = (z[i] + block[(i + 2) % 6]) % 16
        z[i] = s_boxes_b4[i + 6][z[i]]

    t_0 = (z[0] + z[1] + z[2] + block[5]) % 16
    t_1 = (z[1] + z[3] + block[4]) % 16
    t_2 = (z[2] + z[3] + y[0] + block[3]) % 16
    t_3 = (z[3] + z[4] + z[5] + block[1]) % 16
    t_4 = (z[4] + z[0] + block[0]) % 16
    t_5 = (z[5] + z[0] + y[3] + block[2]) % 16
    t   = [t_0, t_1, t_2, t_3, t_4, t_5]

    return HW[t[block_idx]]

t_i_store_locations = [[[0] * 10] * (BLOCK_WIDTH_B4 - 1)] * (KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4 - 2)
for round_idx in range(len(t_i_store_locations), KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4):
    corr_round = []
    for block_idx in range(BLOCK_WIDTH_B4 - 1):
        hyps = np.array([hypothesis_b4_rws_t_i_store(iv, key, round_idx, block_idx) for i, key in enumerate(real_keys) for iv in seeds_varying[i]])
        corr = corr_coef(hyps, traces_varying.reshape((-1, traces_varying.shape[2])))
        if round_idx == 12:
            loc = 16500 + np.argmax(corr[16500:19300])
        elif round_idx == 13:
            loc = 38400 + np.argmax(corr[38400:41400])
        else:
            raise ValueError("Should not happen")
        corr_round.append(list(range(loc - 5, loc + 5)))
        plt.plot(corr)
        plt.ylim([-0.5, 0.5])
        plt.title(f"Round {round_idx}, Block {block_idx}, Location: {loc}")
        plt.xlim([16500, 19300] if round_idx == 12 else [38400, 41400])
        plt.show()
    t_i_store_locations.append(corr_round)
t_i_store_locations = np.array(t_i_store_locations)


In [ ]:
def hypothesis_b4_rws_t_i_during_last_round(iv: str, key: List[int], round_idx: int, block_idx: int) -> int:
    indices, whitening = chacha_random_b4(iv)

    block = [(key[indices[i]] + whitening[i]) % 16 for i in range(BLOCK_WIDTH_B4 * round_idx, BLOCK_WIDTH_B4 * (round_idx + 1))]
    for i in range(3):
        block[2*i + 1] = (block[2*i + 1] + block[2*i]) % 16
    y = [s_boxes_b4[i][block[i]] for i in range(BLOCK_WIDTH_B4 - 1)]
    z = [0] * 6
    for i in range(3):
        z[2*i] = (y[(2*i - 1) % 6] + y[2*i]) % 16
        z[2*i + 1] = (y[(2*i + 4) % 6] + y[2*i + 1]) % 16
    
    for i in range(6):
        z[i] = (z[i] + block[(i + 2) % 6]) % 16
        z[i] = s_boxes_b4[i + 6][z[i]]

    t_0 = (z[0] + z[1] + z[2] + block[5]) % 16
    t_1 = (z[1] + z[3] + block[4]) % 16
    t_2 = (z[2] + z[3] + y[0] + block[3]) % 16
    t_3 = (z[3] + z[4] + z[5] + block[1]) % 16
    t_4 = (z[4] + z[0] + block[0]) % 16
    t_5 = (z[5] + z[0] + y[3] + block[2]) % 16
    t   = [t_0, t_1, t_2, t_3, t_4, t_5]

    return HW[t[block_idx]]

t_i_last_round_locations = [[[0] * 10] * (BLOCK_WIDTH_B4 - 1)] * (KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4 - 2)
for round_idx in range(len(t_i_last_round_locations), KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4):
    corr_round = []
    for block_idx in range(BLOCK_WIDTH_B4 - 1):
        hyps = np.array([hypothesis_b4_rws_t_i_during_last_round(iv, key, round_idx, block_idx) for i, key in enumerate(real_keys) for iv in seeds_varying[i]])
        corr = corr_coef(hyps, traces_varying.reshape((-1, traces_varying.shape[2])))
        if round_idx == 12:
            loc = 19000 + np.argmax(corr[19000:22500])
        elif round_idx == 13:
            loc = 41000 + np.argmax(corr[41000:44500])
        else:
            raise ValueError("Should not happen")
        corr_round.append(list(range(loc - 5, loc + 5)))
        plt.plot(corr)
        plt.ylim([-0.5, 0.5])
        plt.title(f"Round {round_idx}, Block {block_idx}, Location: {loc}")
        plt.xlim([19000, 22500] if round_idx == 12 else [41000, 44500])
        plt.show()
    t_i_last_round_locations.append(corr_round)
t_i_last_round_locations = np.array(t_i_last_round_locations)

In [ ]:
def hypothesis_b4_rws_sboxes_third_round(iv: str, key: List[int], round_idx: int, block_idx: int) -> int:
    indices, whitening = chacha_random_b4(iv)

    block = [(key[indices[i]] + whitening[i]) % 16 for i in range(BLOCK_WIDTH_B4 * round_idx, BLOCK_WIDTH_B4 * (round_idx + 1))]
    for i in range(3):
        block[2*i + 1] = (block[2*i + 1] + block[2*i]) % 16
    y = [s_boxes_b4[i][block[i]] for i in range(BLOCK_WIDTH_B4 - 1)]
    z = [0] * 6
    for i in range(3):
        z[2*i] = (y[(2*i - 1) % 6] + y[2*i]) % 16
        z[2*i + 1] = (y[(2*i + 4) % 6] + y[2*i + 1]) % 16
    
    for i in range(6):
        z[i] = (z[i] + block[(i + 2) % 6]) % 16
        z[i] = s_boxes_b4[i + 6][z[i]]
    
    t_0 = (z[0] + z[1] + z[2] + block[5]) % 16
    t_1 = (z[1] + z[3] + block[4]) % 16
    t_2 = (z[2] + z[3] + y[0] + block[3]) % 16
    t_3 = (z[3] + z[4] + z[5] + block[1]) % 16
    t_4 = (z[4] + z[0] + block[0]) % 16
    t_5 = (z[5] + z[0] + y[3] + block[2]) % 16
    t   = [t_0, t_1, t_2, t_3, t_4, t_5]

    return HW[s_boxes_b4[block_idx + 12][t[block_idx]]]

third_sbox_output_locations = [[[0] * 10] * (BLOCK_WIDTH_B4 - 1)] * (KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4 - 2)
for round_idx in range(len(third_sbox_output_locations), KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4):
    corr_round = []
    for block_idx in range(BLOCK_WIDTH_B4 - 1):
        hyps = np.array([hypothesis_b4_rws_sboxes_third_round(iv, key, round_idx, block_idx) for i, key in enumerate(real_keys) for iv in seeds_varying[i]])
        corr = corr_coef(hyps, traces_varying.reshape((-1, traces_varying.shape[2])))
        loc = np.argmax(corr)
        corr_round.append(list(range(loc - 5, loc + 5)))
        plt.plot(corr)
        plt.ylim([-0.5, 0.5])
        plt.title(f"Round {round_idx}, Block {block_idx}, Location: {loc}")
        plt.show()
    third_sbox_output_locations.append(corr_round)
third_sbox_output_locations = np.array(third_sbox_output_locations)


In [ ]:
hw_x_i_locations = np.array([[[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[ 2035,  2036,  2037,  2038,  2039,  2040,  2041,  2042,  2043,
          2044],
        [ 2512,  2513,  2514,  2515,  2516,  2517,  2518,  2519,  2520,
          2521],
        [ 2839,  2840,  2841,  2842,  2843,  2844,  2845,  2846,  2847,
          2848],
        [ 3166,  3167,  3168,  3169,  3170,  3171,  3172,  3173,  3174,
          3175],
        [ 3494,  3495,  3496,  3497,  3498,  3499,  3500,  3501,  3502,
          3503],
        [ 3822,  3823,  3824,  3825,  3826,  3827,  3828,  3829,  3830,
          3831],
        [ 4014,  4015,  4016,  4017,  4018,  4019,  4020,  4021,  4022,
          4023]],

       [[23881, 23882, 23883, 23884, 23885, 23886, 23887, 23888, 23889,
         23890],
        [24358, 24359, 24360, 24361, 24362, 24363, 24364, 24365, 24366,
         24367],
        [24685, 24686, 24687, 24688, 24689, 24690, 24691, 24692, 24693,
         24694],
        [25013, 25014, 25015, 25016, 25017, 25018, 25019, 25020, 25021,
         25022],
        [25340, 25341, 25342, 25343, 25344, 25345, 25346, 25347, 25348,
         25349],
        [25668, 25669, 25670, 25671, 25672, 25673, 25674, 25675, 25676,
         25677],
        [25860, 25861, 25862, 25863, 25864, 25865, 25866, 25867, 25868,
         25869]]])

hw_x_i_diff_locations = np.array([[[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[ 2220,  2221,  2222,  2223,  2224,  2225,  2226,  2227,  2228,
          2229],
        [ 2549,  2550,  2551,  2552,  2553,  2554,  2555,  2556,  2557,
          2558],
        [ 2875,  2876,  2877,  2878,  2879,  2880,  2881,  2882,  2883,
          2884],
        [ 3201,  3202,  3203,  3204,  3205,  3206,  3207,  3208,  3209,
          3210],
        [ 3529,  3530,  3531,  3532,  3533,  3534,  3535,  3536,  3537,
          3538],
        [ 3857,  3858,  3859,  3860,  3861,  3862,  3863,  3864,  3865,
          3866]],

       [[24066, 24067, 24068, 24069, 24070, 24071, 24072, 24073, 24074,
         24075],
        [24393, 24394, 24395, 24396, 24397, 24398, 24399, 24400, 24401,
         24402],
        [24721, 24722, 24723, 24724, 24725, 24726, 24727, 24728, 24729,
         24730],
        [25048, 25049, 25050, 25051, 25052, 25053, 25054, 25055, 25056,
         25057],
        [25375, 25376, 25377, 25378, 25379, 25380, 25381, 25382, 25383,
         25384],
        [25702, 25703, 25704, 25705, 25706, 25707, 25708, 25709, 25710,
         25711]]])

block_additions_locations = np.array([[[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[ 4951,  4952,  4953,  4954,  4955,  4956,  4957,  4958,  4959,
          4960],
        [ 4585,  4586,  4587,  4588,  4589,  4590,  4591,  4592,  4593,
          4594],
        [ 4477,  4478,  4479,  4480,  4481,  4482,  4483,  4484,  4485,
          4486]],

       [[26798, 26799, 26800, 26801, 26802, 26803, 26804, 26805, 26806,
         26807],
        [26798, 26799, 26800, 26801, 26802, 26803, 26804, 26805, 26806,
         26807],
        [26323, 26324, 26325, 26326, 26327, 26328, 26329, 26330, 26331,
         26332]]])

hw_x_i_during_sbox_locations = np.array([[[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[ 4952,  4953,  4954,  4955,  4956,  4957,  4958,  4959,  4960,
          4961],
        [ 6176,  6177,  6178,  6179,  6180,  6181,  6182,  6183,  6184,
          6185],
        [ 6500,  6501,  6502,  6503,  6504,  6505,  6506,  6507,  6508,
          6509],
        [ 7242,  7243,  7244,  7245,  7246,  7247,  7248,  7249,  7250,
          7251],
        [ 8334,  8335,  8336,  8337,  8338,  8339,  8340,  8341,  8342,
          8343],
        [ 4715,  4716,  4717,  4718,  4719,  4720,  4721,  4722,  4723,
          4724]],

       [[26798, 26799, 26800, 26801, 26802, 26803, 26804, 26805, 26806,
         26807],
        [26797, 26798, 26799, 26800, 26801, 26802, 26803, 26804, 26805,
         26806],
        [28346, 28347, 28348, 28349, 28350, 28351, 28352, 28353, 28354,
         28355],
        [29301, 29302, 29303, 29304, 29305, 29306, 29307, 29308, 29309,
         29310],
        [30181, 30182, 30183, 30184, 30185, 30186, 30187, 30188, 30189,
         30190],
        [26561, 26562, 26563, 26564, 26565, 26566, 26567, 26568, 26569,
         26570]]])

sbox_output_locations = np.array([[[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[ 5278,  5279,  5280,  5281,  5282,  5283,  5284,  5285,  5286,
          5287],
        [ 6052,  6053,  6054,  6055,  6056,  6057,  6058,  6059,  6060,
          6061],
        [ 6827,  6828,  6829,  6830,  6831,  6832,  6833,  6834,  6835,
          6836],
        [ 7993,  7994,  7995,  7996,  7997,  7998,  7999,  8000,  8001,
          8002],
        [ 8344,  8345,  8346,  8347,  8348,  8349,  8350,  8351,  8352,
          8353],
        [ 9163,  9164,  9165,  9166,  9167,  9168,  9169,  9170,  9171,
          9172]],

       [[27124, 27125, 27126, 27127, 27128, 27129, 27130, 27131, 27132,
         27133],
        [27899, 27900, 27901, 27902, 27903, 27904, 27905, 27906, 27907,
         27908],
        [28673, 28674, 28675, 28676, 28677, 28678, 28679, 28680, 28681,
         28682],
        [29839, 29840, 29841, 29842, 29843, 29844, 29845, 29846, 29847,
         29848],
        [30190, 30191, 30192, 30193, 30194, 30195, 30196, 30197, 30198,
         30199],
        [31009, 31010, 31011, 31012, 31013, 31014, 31015, 31016, 31017,
         31018]]])

z_2_i_2_i_1_locations = np.array([[[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[ 9714,  9715,  9716,  9717,  9718,  9719,  9720,  9721,  9722,
          9723],
        [ 9937,  9938,  9939,  9940,  9941,  9942,  9943,  9944,  9945,
          9946],
        [10131, 10132, 10133, 10134, 10135, 10136, 10137, 10138, 10139,
         10140],
        [10726, 10727, 10728, 10729, 10730, 10731, 10732, 10733, 10734,
         10735],
        [11082, 11083, 11084, 11085, 11086, 11087, 11088, 11089, 11090,
         11091],
        [10607, 10608, 10609, 10610, 10611, 10612, 10613, 10614, 10615,
         10616]],

       [[31561, 31562, 31563, 31564, 31565, 31566, 31567, 31568, 31569,
         31570],
        [31783, 31784, 31785, 31786, 31787, 31788, 31789, 31790, 31791,
         31792],
        [31977, 31978, 31979, 31980, 31981, 31982, 31983, 31984, 31985,
         31986],
        [32574, 32575, 32576, 32577, 32578, 32579, 32580, 32581, 32582,
         32583],
        [32928, 32929, 32930, 32931, 32932, 32933, 32934, 32935, 32936,
         32937],
        [32452, 32453, 32454, 32455, 32456, 32457, 32458, 32459, 32460,
         32461]]])

add_pre_second_sbox_output_locations = np.array([[[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[11264, 11265, 11266, 11267, 11268, 11269, 11270, 11271, 11272,
         11273],
        [12246, 12247, 12248, 12249, 12250, 12251, 12252, 12253, 12254,
         12255],
        [13228, 13229, 13230, 13231, 13232, 13233, 13234, 13235, 13236,
         13237],
        [14407, 14408, 14409, 14410, 14411, 14412, 14413, 14414, 14415,
         14416],
        [15252, 15253, 15254, 15255, 15256, 15257, 15258, 15259, 15260,
         15261],
        [16148, 16149, 16150, 16151, 16152, 16153, 16154, 16155, 16156,
         16157]],

       [[33110, 33111, 33112, 33113, 33114, 33115, 33116, 33117, 33118,
         33119],
        [34091, 34092, 34093, 34094, 34095, 34096, 34097, 34098, 34099,
         34100],
        [35075, 35076, 35077, 35078, 35079, 35080, 35081, 35082, 35083,
         35084],
        [36253, 36254, 36255, 36256, 36257, 36258, 36259, 36260, 36261,
         36262],
        [37098, 37099, 37100, 37101, 37102, 37103, 37104, 37105, 37106,
         37107],
        [37994, 37995, 37996, 37997, 37998, 37999, 38000, 38001, 38002,
         38003]]])

second_sbox_output_locations = np.array([[[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[11409, 11410, 11411, 11412, 11413, 11414, 11415, 11416, 11417,
         11418],
        [13047, 13048, 13049, 13050, 13051, 13052, 13053, 13054, 13055,
         13056],
        [14059, 14060, 14061, 14062, 14063, 14064, 14065, 14066, 14067,
         14068],
        [14386, 14387, 14388, 14389, 14390, 14391, 14392, 14393, 14394,
         14395],
        [15398, 15399, 15400, 15401, 15402, 15403, 15404, 15405, 15406,
         15407],
        [16563, 16564, 16565, 16566, 16567, 16568, 16569, 16570, 16571,
         16572]],

       [[33256, 33257, 33258, 33259, 33260, 33261, 33262, 33263, 33264,
         33265],
        [34208, 34209, 34210, 34211, 34212, 34213, 34214, 34215, 34216,
         34217],
        [35905, 35906, 35907, 35908, 35909, 35910, 35911, 35912, 35913,
         35914],
        [36232, 36233, 36234, 36235, 36236, 36237, 36238, 36239, 36240,
         36241],
        [37244, 37245, 37246, 37247, 37248, 37249, 37250, 37251, 37252,
         37253],
        [38409, 38410, 38411, 38412, 38413, 38414, 38415, 38416, 38417,
         38418]]])

t_i_store_locations = np.array([[[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[18479, 18480, 18481, 18482, 18483, 18484, 18485, 18486, 18487,
         18488],
        [17198, 17199, 17200, 17201, 17202, 17203, 17204, 17205, 17206,
         17207],
        [17854, 17855, 17856, 17857, 17858, 17859, 17860, 17861, 17862,
         17863],
        [18509, 18510, 18511, 18512, 18513, 18514, 18515, 18516, 18517,
         18518],
        [18584, 18585, 18586, 18587, 18588, 18589, 18590, 18591, 18592,
         18593],
        [18822, 18823, 18824, 18825, 18826, 18827, 18828, 18829, 18830,
         18831]],

       [[40325, 40326, 40327, 40328, 40329, 40330, 40331, 40332, 40333,
         40334],
        [39044, 39045, 39046, 39047, 39048, 39049, 39050, 39051, 39052,
         39053],
        [39700, 39701, 39702, 39703, 39704, 39705, 39706, 39707, 39708,
         39709],
        [41293, 41294, 41295, 41296, 41297, 41298, 41299, 41300, 41301,
         41302],
        [40430, 40431, 40432, 40433, 40434, 40435, 40436, 40437, 40438,
         40439],
        [40667, 40668, 40669, 40670, 40671, 40672, 40673, 40674, 40675,
         40676]]])

t_i_last_round_locations = np.array([[[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[19447, 19448, 19449, 19450, 19451, 19452, 19453, 19454, 19455,
         19456],
        [19448, 19449, 19450, 19451, 19452, 19453, 19454, 19455, 19456,
         19457],
        [20013, 20014, 20015, 20016, 20017, 20018, 20019, 20020, 20021,
         20022],
        [20578, 20579, 20580, 20581, 20582, 20583, 20584, 20585, 20586,
         20587],
        [21143, 21144, 21145, 21146, 21147, 21148, 21149, 21150, 21151,
         21152],
        [21710, 21711, 21712, 21713, 21714, 21715, 21716, 21717, 21718,
         21719]],

       [[41294, 41295, 41296, 41297, 41298, 41299, 41300, 41301, 41302,
         41303],
        [41293, 41294, 41295, 41296, 41297, 41298, 41299, 41300, 41301,
         41302],
        [41859, 41860, 41861, 41862, 41863, 41864, 41865, 41866, 41867,
         41868],
        [42424, 42425, 42426, 42427, 42428, 42429, 42430, 42431, 42432,
         42433],
        [42990, 42991, 42992, 42993, 42994, 42995, 42996, 42997, 42998,
         42999],
        [43555, 43556, 43557, 43558, 43559, 43560, 43561, 43562, 43563,
         43564]]])

third_sbox_output_locations = np.array([[[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]],

       [[19269, 19270, 19271, 19272, 19273, 19274, 19275, 19276, 19277,
         19278],
        [19834, 19835, 19836, 19837, 19838, 19839, 19840, 19841, 19842,
         19843],
        [20400, 20401, 20402, 20403, 20404, 20405, 20406, 20407, 20408,
         20409],
        [20966, 20967, 20968, 20969, 20970, 20971, 20972, 20973, 20974,
         20975],
        [21350, 21351, 21352, 21353, 21354, 21355, 21356, 21357, 21358,
         21359],
        [22157, 22158, 22159, 22160, 22161, 22162, 22163, 22164, 22165,
         22166]],

       [[41116, 41117, 41118, 41119, 41120, 41121, 41122, 41123, 41124,
         41125],
        [41686, 41687, 41688, 41689, 41690, 41691, 41692, 41693, 41694,
         41695],
        [42246, 42247, 42248, 42249, 42250, 42251, 42252, 42253, 42254,
         42255],
        [42811, 42812, 42813, 42814, 42815, 42816, 42817, 42818, 42819,
         42820],
        [43196, 43197, 43198, 43199, 43200, 43201, 43202, 43203, 43204,
         43205],
        [44002, 44003, 44004, 44005, 44006, 44007, 44008, 44009, 44010,
         44011]]])

In [ ]:
alpha = 0.5

fig = px.line(t_test.statistic)
fig.add_hline(4.5, line_color="red")
fig.add_hline(-4.5, line_color="red")
fig.add_vline(22246, line_color="black")

for block_idx in range(BLOCK_WIDTH_B4):
    fig.add_vline(hw_x_i_locations[12][block_idx][5], line_color="green", opacity=alpha)
    fig.add_vline(hw_x_i_locations[13][block_idx][5], line_color="green", opacity=alpha)

for block_idx in range(BLOCK_WIDTH_B4 // 2):
    fig.add_vline(block_additions_locations[12][block_idx][5], line_color="purple", opacity=alpha)
    fig.add_vline(block_additions_locations[13][block_idx][5], line_color="purple", opacity=alpha)

    fig.add_vline(hw_x_i_during_sbox_locations[12][2 * block_idx][5], line_color="plum", opacity=alpha)
    fig.add_vline(hw_x_i_during_sbox_locations[13][2 * block_idx][5], line_color="plum", opacity=alpha)

for block_idx in range(BLOCK_WIDTH_B4 - 1):
    fig.add_vline(hw_x_i_diff_locations[12][block_idx][5], line_color="lime", opacity=alpha)
    fig.add_vline(hw_x_i_diff_locations[13][block_idx][5], line_color="lime", opacity=alpha)

    fig.add_vline(sbox_output_locations[12][block_idx][5], line_color="red", opacity=alpha)
    fig.add_vline(sbox_output_locations[13][block_idx][5], line_color="red", opacity=alpha)
    
    fig.add_vline(z_2_i_2_i_1_locations[12][block_idx][5], line_color="turquoise", opacity=alpha)
    fig.add_vline(z_2_i_2_i_1_locations[13][block_idx][5], line_color="turquoise", opacity=alpha)

    fig.add_vline(add_pre_second_sbox_output_locations[12][block_idx][5], line_color="orangered", opacity=alpha)
    fig.add_vline(add_pre_second_sbox_output_locations[13][block_idx][5], line_color="orangered", opacity=alpha)

    fig.add_vline(second_sbox_output_locations[12][block_idx][5], line_color="orange", opacity=alpha)
    fig.add_vline(second_sbox_output_locations[13][block_idx][5], line_color="orange", opacity=alpha)

    fig.add_vline(t_i_store_locations[12][block_idx][5], line_color="gray", opacity=alpha)
    fig.add_vline(t_i_store_locations[13][block_idx][5], line_color="gray", opacity=alpha)

    fig.add_vline(t_i_last_round_locations[12][block_idx][5], line_color="gray", opacity=alpha)
    fig.add_vline(t_i_last_round_locations[13][block_idx][5], line_color="gray", opacity=alpha)
    
    fig.add_vline(third_sbox_output_locations[12][block_idx][5], line_color="yellow", opacity=alpha)
    fig.add_vline(third_sbox_output_locations[13][block_idx][5], line_color="yellow", opacity=alpha)

config = {'scrollZoom': True}
#fig.update_layout(dragmode="pan")
fig.show(config=config)